# LetsNet Model for summarization for CASS French Dataset


In [1]:
#Running code for a set of segmented french legal documents


In [14]:
import os
from sentence_transformers import SentenceTransformer
from rouge import Rouge
from sklearn.cluster import KMeans, DBSCAN
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
import json

def evaluate(model_sum, gt_sum):
    rouge = Rouge()
    return rouge.get_scores(model_sum, gt_sum, avg=True)



input_path_cases= os.listdir('testSegmentedCases/')
model_sum = []
true_sum = []
embedder = SentenceTransformer('distiluse-base-multilingual-cased')
root = "testSegmentedCases/"
for file in input_path_cases:
    file_path = os.path.join(root,file)
    if '.txt' in file_path:
        with open(file_path) as f:
            corpus = json.load(f)
        #res = corpus.split("@summary ")
        #text,corpus_summary = res[0],res[1]
        train_sum=""
        corpus_sample = []
        for key,value in corpus.items():
            if key!='Summary':
                corpus_sample.append(value)
        str1 = ''.join(corpus_sample)
        legal_corpus = str1.split(" , ")
        sentence_embeddings = embedder.encode(legal_corpus)
        enc_embedding = sentence_embeddings
        n_clusters = int(np.ceil(len(enc_embedding)**0.5))
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans = kmeans.fit(enc_embedding)
        avg = []
        closest = []
        for j in range(n_clusters):
            idx = np.where(kmeans.labels_ == j)[0]
            avg.append(np.mean(idx))
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,\
                                                   enc_embedding)
        ordering = sorted(range(n_clusters), key=lambda k: avg[k])
        gen_summary = ' '.join([legal_corpus[closest[idx]] for idx in ordering])
        model_sum.append(gen_summary)
        true_sum.append(corpus['Summary'])


print("ROUGE score: {}".format(evaluate(model_sum, true_sum)))

ROUGE score: {'rouge-1': {'f': 0.3722279734021134, 'p': 0.3256664297183214, 'r': 0.5215381566390821}, 'rouge-2': {'f': 0.15983473131337622, 'p': 0.1412333845302405, 'r': 0.22030679362606542}, 'rouge-l': {'f': 0.30207819974032574, 'p': 0.2598302131144868, 'r': 0.40217402469314634}}
